# Modelling a simple RNA circuit

In this notebook, we will simulate the interactions between RNAs in a circuit and how the dynamics of the circuit can be simulated.

In [ ]:
from functools import partial
from src.utils.common.setup_new import construct_circuit_from_cfg
from src.utils.circuit.agnostic_circuits.circuit_manager import CircuitModeller
from src.srv.io.manage.script_manager import script_preamble


In [2]:
config = {}
config['data_path'] = 'data/example/toy_mRNA_circuit.fasta'
config = {
    "data_path": "data/example/toy_mRNA_circuit.fasta",
    "experiment": {
        "purpose": "example"
    },
    "interaction_simulator": {
        "name": "IntaRNA",
        "postprocess": True
    },
    "molecular_params": "src/utils/common/configs/RNA_circuit/molecular_params.json",
    "simulation": {
        "dt": 0.1,
        "t0": 0,
        "t1": 100,
        "solver": "jax"
    },
    "signal": {
        "inputs": ["RNA_0"],
        "outputs": ["RNA_1"],
        "function_name": "step_function",
        "function_kwargs": {
            "impulse_center": 40, 
            "impulse_halfwidth": 5, 
            "target": 0.5
        } 
    },
    "system_type": "RNA"
}

config, data_writer = script_preamble(config=config, data_writer=None, alt_cfg_filepath=None)

In [3]:
circuit = construct_circuit_from_cfg(config_file=config, extra_configs=None)
circuit.__dict__

modeller = CircuitModeller(result_writer=data_writer, config=config)
circuit = modeller.init_circuit(circuit=circuit)
circuit = modeller.simulate_signal(circuit=circuit, signal=circuit.signal, solver=config['simulation']['solver'])
modeller.write_results(circuit)


{'name': 'toy_mRNA_circuit',
 'result_collector': <src.utils.results.results.ResultCollector at 0x7f89fc746fb0>,
 'model': <bioreaction.model.data_containers.BasicModel at 0x7f8998c18a60>,
 'reactions': <bioreaction.model.data_containers.QuantifiedReactions at 0x7f8998c19030>,
 'interactions': <src.srv.parameter_prediction.interactions.MolecularInteractions at 0x7f8998c1ba00>,
 'interactions_state': 'uninitialised',
 'graph': <src.utils.circuit.agnostic_circuits.circuit.Graph at 0x7f89fc7472e0>,
 'circuit_size': 9}